<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DEEPSEEK_OCR_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://deepseek.ai/blog/deepseek-ocr-context-compression

https://github.com/deepseek-ai/DeepSeek-OCR/tree/main

In [1]:
!nvidia-smi

Sat Oct 25 16:35:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   33C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-OCR.git

In [ ]:
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118 -q
!pip install -r /content/DeepSeek-OCR/requirements.txt -q
!pip install flash-attn==2.7.3 --no-build-isolation -q

In [4]:
!pip show flash-attn

Name: flash-attn
Version: 2.7.3
Summary: Flash Attention: Fast and Memory-Efficient Exact Attention
Home-page: https://github.com/Dao-AILab/flash-attention
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: einops, torch
Required-by: 


In [ ]:
# This is a general command, may not work for all versions
# The initial simple reinstall is more likely to auto-detect and work
!pip install flash-attn --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import os

# Set GPU visibility (optional in Colab, but harmless)
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR'

# This should now download a fresh, compatible version of the custom code
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name,
    _attn_implementation='flash_attention_2',
    trust_remote_code=True,
    use_safetensors=True
)
model = model.eval().cuda().to(torch.bfloat16)

print("Model loaded successfully!")

In [7]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '/content/DeepSeek-OCR/assets/show1.jpg'
output_path = '/content/assets1/'

res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Cal

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])
<|ref|>text<|/ref|><|det|>[[20, 30, 330, 40]]<|/det|>
1. 已知：△ABC的两条高BD、CE交于点F，点M、N、分别是AF、BC的中点，连接ED、MN；  

<|ref|>text<|/ref|><|det|>[[20, 42, 130, 50]]<|/det|>
(1)证明：MN垂直平分ED；  

<|ref|>text<|/ref|><|det|>[[20, 52, 300, 61]]<|/det|>
(2)若 \(\angle EBD = \angle DCE = 45^{\circ}\) ，判断以M、E、N、D为顶点的四边形的形状，并证明你的结论；  

<|ref|>image<|/ref|><|det|>[[264, 88, 339, 150]]<|/det|>  

<|ref|>text<|/ref|><|det|>[[20, 209, 333, 228]]<|/det|>
2. 四边形ABCD是正方形，△BEF是等腰直角三角形， \(\angle BEF = 90^{\circ}\) ，BE=EF，连接DF、G为DF的中点，连接EG、CG、EC；  

<|ref|>text<|/ref|><|det|>[[20, 230, 300, 239]]<|/det|>
(1)如图1，若点E在CD边的延长线上，直接写出EG与GC的位置关系及 \(\frac{EC}{GC}\) 的值；  

<|ref|>text<|/ref|><|det|>[[20, 241, 333, 260]]<|/det|>
(2)将图1中的△BEF绕点B顺时针旋转至图2所示位置，请用(1)中所得结论是否仍然成立？若成立，请写出证明过程；若不成立，请说明理由；  

<|ref|>text<|/ref|><|det|>[[20, 262, 333, 272]]<|/det|>
(3)将图1中的△BEF绕点B顺时针旋转 \(\alpha (0^{\circ}< \alpha < 90^{\circ})\) ，若BE=1，AB=√2，当E、F、D三点共线时，求DF的长；  

<|ref|

other: 100%|██████████| 26/26 [00:00<00:00, 108401.50it/s]


In [8]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '/content/DeepSeek-OCR/assets/show2.jpg'
output_path = '/content/assets2/'

res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
<|ref|>sub_title<|/ref|><|det|>[[50, 30, 300, 46]]<|/det|>
# Macro news and views 

<|ref|>sub_title<|/ref|><|det|>[[50, 49, 430, 59]]<|/det|>
## We provide a brief snapshot on the most important economies for the global markets 

<|ref|>sub_title<|/ref|><|det|>[[50, 65, 250, 73]]<|/det|>
### Latest GS proprietary datapoints/major changes in views 

<|ref|>text<|/ref|><|det|>[[50, 72, 250, 95]]<|/det|>
- We now assume a 10pc increase in the US effective tariff rate (vs. 6-8pc prior as reciprocal tariffs and further increases in product-specific tariffs now seem likely). 

<|ref|>text<|/ref|><|det|>[[50, 94, 250, 140]]<|/det|>
- We raised our Dec 2025 core PCE inflation forecast to -3% from 2.5%, yoy, lowered our 2026 CPI growth forecast to 1.7% from 2.4%, GDP growth rate fell below consensus call in 2.5 years—and slightly raised our end-2025 unemployment rate forecast to 4.2% from 4.1% and our 12m recession odds to 

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


yoy, lowered our 2026 CPI growth forecast to 1%) (from jhg. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhi. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhj. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhk. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhl. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhm. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhn. yoy, lowered our 2026 CPI growth forecast to 1%) (from jho. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhp. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhq. yoy, lowered our 2026 CPI growth forecast to 1%) (fr.yoy, lowered our 2026 CPI growth forecast to 1%) (from jhs. yoy, lowered our 2026 CPI growth forecast to 1%) (from jht. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhu. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhv. yoy, lowered our 2026 CPI growth forecast to 1%) (from jhw. yoy, lowered our 2026 CPI growth forecast to 1%

other: 100%|██████████| 42/42 [00:00<00:00, 37465.07it/s]


In [9]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '/content/DeepSeek-OCR/assets/show3.jpg'
output_path = '/content/assets3/'

res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
<|ref|>image<|/ref|><|det|>[[45, 0, 456, 404]]<|/det|>

<|ref|>text<|/ref|><|det|>[[45, 380, 293, 404]]<|/det|>
Cristina Gilanders, PhD, is a researcher at the FPG Child Development Institute at the University of North Carolina-Chapel Hill. She was an investigator in the Nuestros Ninos study, and has worked with dual language learners as a bilingual preschool teacher, teacher educator, and researcher. cristina.gilanders@unc.edu

<|ref|>text<|/ref|><|det|>[[45, 405, 293, 432]]<|/det|>
Dina C. Castro, PhD, is a senior scientist at the FPG Child Development Institute. She was the principal investigator for the Nuestros Ninos study. Her research focuses on improving the quality of early education for children from diverse cultural and linguistic backgrounds. dina.castro@unc.edu

<|ref|>text<|/ref|><|det|>[[55, 433, 293, 440]]<|/det|>
Photos courtesy of the authors.

<|ref|>text<|/ref|><|det|>[[55, 441, 293, 448]]<|/det|

other: 100%|██████████| 39/39 [00:00<00:00, 63822.81it/s]


In [10]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = '/content/DeepSeek-OCR/assets/show4.jpg'
output_path = '/content/assets4/'

res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])
<|ref|>text<|/ref|><|det|>[[12, 20, 390, 40]]<|/det|>
<image>InLocate<|refl>11- 2=<|/refl> in the image.

<|ref|>image<|/ref|><|det|>[[12, 50, 394, 298]]<|/det|>

<|ref|>text<|/ref|><|det|>[[12, 320, 437, 340]]<|/det|>
<image>InLocate<|refl>the teacher</|refl> in the image.

<|ref|>image<|/ref|><|det|>[[12, 352, 394, 590]]<|/det|>

<|ref|>text<|/ref|><|det|>[[72, 590, 379, 600]]<|/det|>
"I like how Bonnie's not stepping on my last nerve."

<|ref|>text<|/ref|><|det|>[[12, 644, 168, 664]]<|/det|>
<image>In这是一张

<|ref|>image<|/ref|><|det|>[[12, 675, 220, 984]]<|/det|>

<|ref|>text<|/ref|><|det|>[[230, 730, 355, 940]]<|/det|>
照片，展示了一辆红色的消防栓。消防栓上有一个笑脸，显得非常友好和亲切。消防栓的顶部有一个黑色的盖子，周围有一些金属铆钉。在消防栓的底部，有一个粉红色的贴纸，上面写着“bowtie”。背景中可以看到一条街道，街道上有几辆停放的汽车和一些树木。整体画面给人一种温馨和友好的感觉。

<|ref|>text<|/ref|><|det|>[[440, 20, 722, 40]]<|/det|>
<image>InDescribe this image in detail.

<|ref|>image<|/ref|><|det|>[[420, 50, 732, 298]]<|/det|>

<|ref|

other: 100%|██████████| 24/24 [00:00<00:00, 108007.83it/s]
